# Serving a Model as an HTTPS Endpoint

We need to understand the application and business context to choose between real-time and batch predictions. Are we trying to optimize for latency or throughput? Does the application require our models to scale automatically throughout the day to handle cyclic traffic requirements? Do we plan to compare models in production through A/B tests?

If our application requires low latency, then we should deploy the model as a real-time API to provide super-fast predictions on single prediction requests over HTTPS. We can deploy, scale, and compare our model prediction servers with SageMaker Endpoints. 

<img src="img/sagemaker-architecture.png" width="80%" align="left">

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
%store -r training_job_name

In [3]:
print(training_job_name)

tensorflow-training-2020-08-10-04-05-18-855


# Copy the Model to the Notebook

In [4]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-east-1-835319576252/tensorflow-training-2020-08-10-04-05-18-855/output/model.tar.gz to ./model.tar.gz


In [5]:
!tar -xvzf ./model.tar.gz

code/
code/inference.py
tensorboard/
metrics/
metrics/confusion_matrix.png
transformers/
transformers/fine-tuned/
transformers/fine-tuned/config.json
transformers/fine-tuned/tf_model.h5
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/assets/
tensorflow/saved_model/0/saved_model.pb
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.data-00000-of-00001
tensorflow/saved_model/0/variables/variables.index


In [6]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

2020-08-11 04:14:08.197025: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/efa/lib:/opt/amazon/efa/lib:/opt/amazon/efa/lib64:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:
2020-08-11 04:14:08.197106: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/ef

# Show `inference.py`

In [7]:
!pygmentize ./code/inference.py

import json
import subprocess
import sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
import tensorflow as tf
from transformers import DistilBertTokenizer

classes=[1, 2, 3, 4, 5]

max_seq_length=128

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def input_handler(data, context):
    transformed_instances = []

    print('DATA {}'.format(data))

    for instance in data:
        data_str = instance.decode('utf-8')
        print('DATA_STR {}'.format(data_str))
        
        tokens = tokenizer.tokenize(data_str)
        print('TOKENS {}'.format(tokens))

        encode_plus_tokens = tokenizer.encode_plus(data_str,
                                                   pad_to_max_length=True,
                                                   max_length=max_seq_length)

        # Convert the text-based tokens to ids from the pre-traine

# Deploy the Model
This will create a default `EndpointConfig` with a single model.  

The next notebook will demonstrate how to perform more advanced `EndpointConfig` strategies to support canary rollouts and A/B testing.

_Note:  If not using a US-based region, you may need to adapt the container image to your current region using the following table:_

https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html

In [8]:
import time
timestamp = int(time.time())

tensorflow_model_name = '{}-{}-{}'.format(training_job_name, 'tfserving', timestamp)

print(tensorflow_model_name)

tensorflow-training-2020-08-10-04-05-18-855-tfserving-1597119253


In [9]:
from sagemaker.tensorflow.serving import Model

tensorflow_model = Model(name=tensorflow_model_name,
                         model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
                         role=role,                
                         framework_version='2.1.0')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [10]:
tensorflow_endpoint_name = '{}-{}-{}'.format(training_job_name, 'tfserving', timestamp)

print(tensorflow_endpoint_name)

tensorflow-training-2020-08-10-04-05-18-855-tfserving-1597119253


In [11]:
tensorflow_model = tensorflow_model.deploy(endpoint_name=tensorflow_endpoint_name,
                                           initial_instance_count=1, # Should use >=2 for high(er) availability 
                                           instance_type='ml.c5.9xlarge', # requires enough disk space for tensorflow, transformers, and bert downloads
                                           wait=False)

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: 1 validation error detected: Value 'tensorflow-training-2020-08-10-04-05-18-855-tfserving-1597119253' at 'modelName' failed to satisfy constraint: Member must have length less than or equal to 63

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, tensorflow_endpoint_name)))


In [ ]:
waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=tensorflow_endpoint_name)

# _Wait Until the ^^ Endpoint ^^ is Deployed_

# Simulate a Prediction from an Application

In [ ]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name=tensorflow_endpoint_name,
                      sagemaker_session=sess,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

# Predict the `star_rating` with Ad Hoc `review_body` Samples

In [ ]:
reviews = ["This is great!"]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

# Predict the `star_rating` with `review_body` Samples from our TSV's

In [ ]:
import csv

df_reviews = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz', 
                         delimiter='\t', 
                         quoting=csv.QUOTE_NONE,
                         compression='gzip')
df_sample_reviews = df_reviews[['review_body', 'star_rating']].sample(n=100)
df_sample_reviews = df_sample_reviews.reset_index()
df_sample_reviews.shape

In [ ]:
import pandas as pd

def predict(review_body):
    return predictor.predict([review_body])[0]

df_sample_reviews['predicted_class'] = df_sample_reviews['review_body'].map(predict)
df_sample_reviews

# Save for Next Notebook(s)

In [ ]:
%store tensorflow_model_name

In [ ]:
%store tensorflow_endpoint_name 

In [ ]:
%store

# Delete Endpoint
To save cost, we should delete the endpoint.

In [ ]:
# sm.delete_endpoint(
#      EndpointName=tensorflow_endpoint_name
# )

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();